The following python code will create a STAC catalog from a single local geotif raster. It will create a folder called 'stac', this is where the json files will be put. 


How do I loop it so it will create STAC items for several rasters in a folder?  

In [ ]:
%pip install rasterio shapely pystac

This code will create many items from a folder

In [1]:
from pathlib import Path
import pystac
import os

catalog = pystac.Catalog(
    id='Drone_Imagery',
    description='Drone Imagery of SRER',
    stac_extensions=['https://stac-extensions.github.io/projection/v1.0.0/schema.json']
)

In [2]:
from shapely.geometry import Polygon, mapping
import rasterio

def get_bbox_and_footprint(dataset):

    # create the bounding box it will depend if it comes from rasterio or rioxarray
    bounds = dataset.bounds

    if isinstance(bounds, rasterio.coords.BoundingBox):
        bbox = [bounds.left, bounds.bottom, bounds.right, bounds.top]
    else:
        bbox = [float(f) for f in bounds()]

    # create the footprint
    footprint = Polygon([
        [bbox[0], bbox[1]],
        [bbox[0], bbox[3]],
        [bbox[2], bbox[3]],
        [bbox[2], bbox[1]]
    ])

    return bbox, mapping(footprint)

    return bbox, mapping(footprint)

In [4]:

from rasterio.warp import transform_bounds, transform_geom

# get a list of files
folder = Path('/home/jgillan/Documents/STAC_drone')
masks = list(folder.rglob('*.tif'))

for mask in masks:

    # open the mask with rasterio
    ds = rasterio.open(mask)

    bbox, footprint = get_bbox_and_footprint(ds)
    
    idx = mask.stem[:10]
    
    from datetime import datetime
    item = pystac.Item(id=idx,
                 geometry=footprint,
                 bbox=bbox,
                 datetime=datetime.utcnow(),
                 properties={})
        
    
    catalog.add_item(item)

print(len(list(catalog.get_items())))
catalog.describe()

3
* <Catalog id=Paranapanema>
  * <Item id=1a_g2_orth>
  * <Item id=1a_g2_sept>
  * <Item id=1a_u6_orth>


In [5]:
from pystac.extensions.projection import AssetProjectionExtension

for mask in masks:
    idx = mask.stem[:10]
    item = catalog.get_item(idx)

    # as before, let's open the mask with rasterio and get bbox and footprint
    ds = rasterio.open(mask)
    bbox, footprint = get_bbox_and_footprint(ds)

    item.add_asset(
        key='watermask',
        asset=pystac.Asset(
            href=mask.as_posix(),
            media_type=pystac.MediaType.GEOTIFF
        )
    )

In [9]:
stac_directory="/home/jgillan/Documents/STAC_drone"

catalog.normalize_hrefs(os.path.join(stac_directory, 'stac_test3'))

catalog.make_all_asset_hrefs_relative()
catalog.save(catalog_type=pystac.CatalogType.SELF_CONTAINED)